In [18]:
import pandas as pd
import numpy as np
import re
import pickle
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')

In [15]:
df = pd.read_csv('./src/train.csv')
print('리뷰 학습 데이터 크기:',df.shape)

리뷰 학습 데이터 크기: (50000, 2)


In [16]:
def score_evaluate(score):
    if score >= 3.5:
        return 1
    else:
        return 0
df['score'] = df['score'].apply(score_evaluate)
df.head()

,review,score
0,오늘 오니기리와 이규동 가서 돈가스랑 계란규동 먹었는데 진짜 합리적인 가격에 호강했...,1
1,여기는 미스터파파 임실치즈피자 인가 그렇습니다 저도 낚였네요 피자는 가격 저렴한 만...,0
2,숯불닭갈비가 그런맛이였다니 처음 먹어본 닭갈비 진짜맛있었어요 춘천또가보고싶네요 정말강추,1
3,바하커피샵 전체적인 분위기는 작고 부담이 없다고 할까 하지만 커피맛이 좋은집 카페인...,0
4,페북에서 봤는데 가관 장사 망,0
...,...,...
49995,하얀짜장 맛있어요 짬뽕도 얼큰하고 든든하게 먹고갑니다,1
49996,친절하시고 반찬도 많아 좋았어요 직접 산에서 해오신 나물들이라고 설명듣고 먹어보니 ...,1
49997,옛날에 좋은 추억이 있어 오랫만에 방문했는데 많이 변했더라구요 카페분위기도 어수선하...,0
49998,저는 되게 맛있게 먹고왔어서요 부산에 가본 횟집들 부산이라는 이름만 있지 막상 가보...,1


In [17]:
df['review'] = df['review'].str.replace('[^ 가-힣]',' ').str.replace(' +',' ')
df.head()

,review,score
0,오늘 오니기리와 이규동 가서 돈가스랑 계란규동 먹었는데 진짜 합리적인 가격에 호강했...,1
1,여기는 미스터파파 임실치즈피자 인가 그렇습니다 저도 낚였네요 피자는 가격 저렴한 만...,0
2,숯불닭갈비가 그런맛이였다니 처음 먹어본 닭갈비 진짜맛있었어요 춘천또가보고싶네요 정말강추,1
3,바하커피샵 전체적인 분위기는 작고 부담이 없다고 할까 하지만 커피맛이 좋은집 카페인...,0
4,페북에서 봤는데 가관 장사 망,0


In [5]:
from konlpy.tag import Okt

tag_list = ['Noun','Verb','Adjective','VerbPrefix']
okt = Okt()
tokenized_data = []  
for i in range(df.shape[0]):
    tokenized_sentence = okt.pos(df['review'][i], stem=True) # 토큰화
    tag_checked_sentence = []
    for j in tokenized_sentence:
        x,y = j
        if y in tag_list:
            tag_checked_sentence.append(x)
    print(f'\r{i+1}개 형태소분석 완료.',end='')
    tokenized_data.append(tag_checked_sentence)
df['토큰화댓글'] = tokenized_data
df.head()

50000개 형태소분석 완료.

,review,score,토큰화댓글
0,오늘 오니기리와 이규동 가서 돈가스랑 계란규동 먹었는데 진짜 합리적인 가격에 호강했...,1,"[오늘, 오니기리, 이규동, 가다, 돈가스, 계란, 규동, 먹다, 진짜, 합리, 가..."
1,여기는 미스터파파 임실치즈피자 인가 그렇습니다 저도 낚였네요 피자는 가격 저렴한 만...,0,"[여기다, 미스터, 파파, 임실, 치즈, 피자, 그렇다, 저, 낚였네, 피자, 가격..."
2,숯불닭갈비가 그런맛이였다니 처음 먹어본 닭갈비 진짜맛있었어요 춘천또가보고싶네요 정말강추,1,"[숯불, 닭갈비, 맛, 이다, 처음, 먹다, 보다, 닭갈비, 진짜, 맛있다, 춘천,..."
3,바하커피샵 전체적인 분위기는 작고 부담이 없다고 할까 하지만 커피맛이 좋은집 카페인...,0,"[바하, 커피샵, 전체, 분위기, 작고, 부담, 없다, 하다, 커피, 맛, 좋다, ..."
4,페북에서 봤는데 가관 장사 망,0,"[페북, 보다, 가관, 장사, 망]"
...,...,...,...
49995,하얀짜장 맛있어요 짬뽕도 얼큰하고 든든하게 먹고갑니다,1,"[하얗다, 짜장, 맛있다, 짬뽕, 얼큰하다, 든든하다, 먹다, 가다]"
49996,친절하시고 반찬도 많아 좋았어요 직접 산에서 해오신 나물들이라고 설명듣고 먹어보니 ...,1,"[친절하다, 반찬, 많다, 좋다, 직접, 산, 해오다, 나물, 설명, 듣다, 먹다,..."
49997,옛날에 좋은 추억이 있어 오랫만에 방문했는데 많이 변했더라구요 카페분위기도 어수선하...,0,"[옛날, 좋다, 추억, 있다, 만, 방문, 하다, 변하다, 카페, 분위기, 어수선하..."
49998,저는 되게 맛있게 먹고왔어서요 부산에 가본 횟집들 부산이라는 이름만 있지 막상 가보...,1,"[저, 맛있다, 먹다, 오다, 부산, 가보다, 횟집, 부산, 이름, 있다, 막상, ..."


In [13]:
df = df[df['토큰화댓글'].str.len() > 3]
df = df.reset_index(drop=True) 
print('리뷰 학습 데이터 크기:',df.shape)
df.head()

리뷰 학습 데이터 크기: (48817, 3)


,review,score,토큰화댓글
0,오늘 오니기리와 이규동 가서 돈가스랑 계란규동 먹었는데 진짜 합리적인 가격에 호강했...,1,"[오늘, 오니기리, 이규동, 가다, 돈가스, 계란, 규동, 먹다, 진짜, 합리, 가..."
1,여기는 미스터파파 임실치즈피자 인가 그렇습니다 저도 낚였네요 피자는 가격 저렴한 만...,0,"[여기다, 미스터, 파파, 임실, 치즈, 피자, 그렇다, 저, 낚였네, 피자, 가격..."
2,숯불닭갈비가 그런맛이였다니 처음 먹어본 닭갈비 진짜맛있었어요 춘천또가보고싶네요 정말강추,1,"[숯불, 닭갈비, 맛, 이다, 처음, 먹다, 보다, 닭갈비, 진짜, 맛있다, 춘천,..."
3,바하커피샵 전체적인 분위기는 작고 부담이 없다고 할까 하지만 커피맛이 좋은집 카페인...,0,"[바하, 커피샵, 전체, 분위기, 작고, 부담, 없다, 하다, 커피, 맛, 좋다, ..."
4,페북에서 봤는데 가관 장사 망,0,"[페북, 보다, 가관, 장사, 망]"


In [24]:
df = pd.read_csv('./src/train_token.csv')

In [25]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=40000, oov_token = True)
tokenizer.fit_on_texts(df['토큰화댓글'])
df['토큰'] = tokenizer.texts_to_sequences(df['토큰화댓글'])
df.head()

,review,score,토큰화댓글,토큰
0,오늘 오니기리와 이규동 가서 돈가스랑 계란규동 먹었는데 진짜 합리적인 가격에 호강했...,1,"['오늘', '오니기리', '이규동', '가다', '돈가스', '계란', '규동',...","[135, 7564, 16042, 7, 770, 1450, 16043, 3, 19,..."
1,여기는 미스터파파 임실치즈피자 인가 그렇습니다 저도 낚였네요 피자는 가격 저렴한 만...,0,"['여기다', '미스터', '파파', '임실', '치즈', '피자', '그렇다', ...","[127, 3365, 9170, 5242, 218, 118, 37, 80, 8224..."
2,숯불닭갈비가 그런맛이였다니 처음 먹어본 닭갈비 진짜맛있었어요 춘천또가보고싶네요 정말강추,1,"['숯불', '닭갈비', '맛', '이다', '처음', '먹다', '보다', '닭갈...","[717, 595, 4, 14, 75, 3, 15, 595, 19, 5, 976, ..."
3,바하커피샵 전체적인 분위기는 작고 부담이 없다고 할까 하지만 커피맛이 좋은집 카페인...,0,"['바하', '커피샵', '전체', '분위기', '작고', '부담', '없다', '...","[8225, 16044, 741, 91, 1173, 721, 8, 2, 156, 4..."
4,페북에서 봤는데 가관 장사 망,0,"['페북', '보다', '가관', '장사', '망']","[4017, 15, 2136, 72, 2523]"


In [26]:
import pickle
with open('./src/tokenizer.pickle', 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [27]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from sklearn.preprocessing import LabelEncoder

train = pad_sequences(df['토큰'], maxlen=20)
label = df['score']
encoder = LabelEncoder()
batch_size = label.shape[0]
input_dim = 1
label = encoder.fit_transform(label)
label = np.reshape(label, (batch_size, input_dim))    
model = Sequential()
model.add(Embedding(40000, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])    
hist = model.fit(train, label, batch_size=32, epochs=5)

Epoch 1/5
1526/1526 [==============================] - 222s 143ms/step - loss: 0.3000 - accuracy: 0.8829
Epoch 2/5
1526/1526 [==============================] - 193s 127ms/step - loss: 0.2244 - accuracy: 0.9195
Epoch 3/5
1526/1526 [==============================] - 225s 147ms/step - loss: 0.1828 - accuracy: 0.9356
Epoch 4/5
1526/1526 [==============================] - 209s 137ms/step - loss: 0.1473 - accuracy: 0.9480
Epoch 5/5
1526/1526 [==============================] - 204s 133ms/step - loss: 0.1154 - accuracy: 0.9594


In [28]:
model.save('./src/model.h5')